In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import applications
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import time
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

batch_size = 16
num_classes = 10
epochs = 5
data_augmentation = False
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

Using TensorFlow backend.
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

# Parse do dataset e separação entre conjuntos de teste e treinamento

In [2]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_train.shape[1:])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(32, 32, 3)


# Testamos com um modelo de rede neural com 4 convoluções em 2D
#### Definimos uma rede neural simples para obter boa comparação com a rede neural profunda, testada a seguir

In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Teste do modelo construido manualmente, é possível observar uma acurácia de validação de 63%. Comparando aos resultados proximos de competições, o modelo obteve acurácia baixa.

In [4]:
start_time = time.time()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)
print("Fitting duration: " + str(time.time() - start_time))

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - -10767s -215337us/step - loss: 1.7322 - accuracy: 0.3642 - val_loss: 1.4272 - val_accuracy: 0.4789
Epoch 2/5
50000/50000 [==============================] - 30s 598us/step - loss: 1.4108 - accuracy: 0.4911 - val_loss: 1.2931 - val_accuracy: 0.5278
Epoch 3/5
50000/50000 [==============================] - 29s 590us/step - loss: 1.2676 - accuracy: 0.5497 - val_loss: 1.1410 - val_accuracy: 0.6009
Epoch 4/5
50000/50000 [==============================] - 30s 591us/step - loss: 1.1688 - accuracy: 0.5846 - val_loss: 1.0399 - val_accuracy: 0.6357
Epoch 5/5
50000/50000 [==============================] - 29s 579us/step - loss: 1.0897 - accuracy: 0.6173 - val_loss: 1.0185 - val_accuracy: 0.6437
Fitting duration: -10644.860817670822
10000/10000 [==============================] - 1s 135us/step
Test loss: 1.0184743337631226
Test accuracy: 0.6437000036239624


# Com o intuito de identificar o comportamento de redes profundas para o reconhecimento de padrões, testamos uma rede neural residual. O classificador será baseado na rede ResNet50V2, uma rede residual com 50 camadas.
#### A rede profunda escolhida foi a ResNet devido a sua capacidade de identificação de atributos globais nas imagens, o que pode facilitar a classificação de objetos com formatos e contextos diferentes.

In [7]:
img_height = img_width = 32

# Instanciando o modelo Resnet50V2
base_model = applications.resnet_v2.ResNet50V2(weights=None, include_top=False, input_shape=(img_height, img_width, 3))

# Aplicando pooling e dropout no output
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)

# Gerando camada de output e instanciando o modelo
predictions = Dense(num_classes, activation='softmax')(x)
model2 = Model(inputs=base_model.input, outputs=predictions)

from keras.optimizers import SGD, Adam

model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
start_time = time.time()

model2.fit(x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(x_test, y_test),
      shuffle=True)
print("Fitting duration: " + str(time.time() - start_time))

acc = model2.evaluate(x_test, y_test)
print("Test accuracy:" + str(acc[1]))
print("Loss:" + str(acc[0]))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 366s 7ms/step - loss: 2.3828 - accuracy: 0.2712 - val_loss: 1.7033 - val_accuracy: 0.4109
Epoch 2/5
50000/50000 [==============================] - 366s 7ms/step - loss: 1.7298 - accuracy: 0.4191 - val_loss: 1.5503 - val_accuracy: 0.4905
Epoch 3/5
50000/50000 [==============================] - 373s 7ms/step - loss: 1.4785 - accuracy: 0.4972 - val_loss: 1.6820 - val_accuracy: 0.4585
Epoch 4/5
50000/50000 [==============================] - 384s 8ms/step - loss: 1.3072 - accuracy: 0.5522 - val_loss: 1.2296 - val_accuracy: 0.5829
Epoch 5/5
50000/50000 [==============================] - 378s 8ms/step - loss: 1.1807 - accuracy: 0.5991 - val_loss: 1.2092 - val_accuracy: 0.6138
Fitting duration: 1876.0506908893585
10000/10000 [==============================] - 10s 963us/step
Test accuracy:0.6137999892234802
Loss:1.2092481250762939


## Mesmo com um tempo de treinamento muito maior e uma estrutura mais complexa, a rede RESNET obteve resultado muito proximo á rede neural criada anteriormente, isso pode ser explicado por:
### A rede RESNET necessita de mais tempo de treinamento, devido a quantidade muito maior de camadas.
### Possivelmente essa acurácia melhoraria com mais épocas em seu treinamento.

In [ ]:
model2.metrics_names